In [0]:
%pip install -qqq -r requirements.txt

In [0]:
import yaml

with open('./config.yaml', 'r') as file:
    config = yaml.safe_load(file)
  
#load global configs
databricks_config = config['databricks_config']

#load uc configs
catalog=databricks_config['catalog']
schema=databricks_config['schema']

In [0]:
def run_job() -> str:
  """
  A function that runs a specified job in the function code

  Args:
    None

  Returns:
    dict: result of the run serialized as a dictionary
  """
  import json
  from databricks.sdk import WorkspaceClient
  from databricks_ai_bridge import ModelServingUserCredentials
  # Initialize Databricks Workspace Client
  w = WorkspaceClient(credentials_strategy=ModelServingUserCredentials())

  run_response = w.jobs.run_now(job_id="898077102753814")
  return json.dumps(run_response.result().as_dict())

In [0]:
out = run_job()
print(out)

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()

function_info = client.create_python_function(
    func=run_job,
    catalog=catalog,
    schema=schema,
    replace=True,
    dependencies=[
        "databricks-sdk>=0.40.0",
        "pydantic>=2.0.0",
        "unitycatalog-ai[databricks]",
        "unitycatalog-langchain[databricks]",
        "langgraph==0.3.4",
        "databricks-langchain==0.5.1",
        "mlflow[databricks]",
    ],
    environment_version="4",
)

In [0]:
#test function in notebook
import os
from dbruntime.databricks_repl_context import get_context

HOSTNAME = get_context().browserHostName
USERNAME = get_context().user
TOKEN = get_context().apiToken

# os.environ['DATABRICKS_TOKEN'] = dbutils.secrets.get(scope="doan", key="db-pat-token")
os.environ['DATABRICKS_TOKEN'] = TOKEN
os.environ['DATABRICKS_HOST'] = "https://"+HOSTNAME

result = client.execute_function(
  function_name=f"{catalog}.{schema}.run_job",
)
print(result)